# 函数进阶

## 函数是基本类型

在 `Python` 中，函数是一种基本类型的对象，这意味着

- 可以将函数作为参数传给另一个函数
- 将函数作为字典的值储存
- 将函数作为另一个函数的返回值

In [1]:
def square(x):
    """Square of x."""
    return x*x

def cube(x):
    """Cube of x."""
    return x*x*x

作为字典的值：

In [5]:
funcs = {
    'square': square,
    'cube': cube,}

x = 2
print(square(x))
print(cube(x))

for func in sorted(funcs):
    print(func, funcs[func](x))

4
8
cube 8
square 4


## 函数参数

### 引用传递

`Python` 中的函数传递方式是 `call by reference` 即引用传递，例如，对于这样的用法：

    x = [10, 11, 12]
    f(x)

传递给函数 `f` 的是一个指向 `x` 所包含内容的引用，如果我们修改了这个引用所指向内容的值（例如 `x[0]=999`），那么外面的 `x` 的值也会被改变。不过如果我们在函数中赋给 `x` 一个新的值（例如另一个列表），那么在函数外面的 `x` 的值不会改变：

In [6]:
def mod_f(x):
    x[0] = 999
    return x

x = [1, 2, 3]

print(x)
print(mod_f(x))
print(x)

[1, 2, 3]
[999, 2, 3]
[999, 2, 3]


In [7]:
def no_mod_f(x):
    x = [4, 5, 6]
    return x

x = [1,2,3]

print(x)
print(no_mod_f(x))
print(x)

[1, 2, 3]
[4, 5, 6]
[1, 2, 3]


上面两种不同的原因在【python基本编程】章节有具体介绍。

### 默认参数是可变的！

函数可以传递默认参数，默认参数的绑定发生在函数定义的时候，以后每次调用默认参数时都会使用同一个引用。

这样的机制会导致这种情况的发生：

In [10]:
def f(x = []):
    x.append(1)
    return x

理论上说，我们希望调用 `f()` 时返回的是 `[1]`， 但事实上：

In [11]:
print(f())
print(f())
print(f())
print(f(x = [9,9,9]))
print(f())
print(f())

[1]
[1, 1]
[1, 1, 1]
[9, 9, 9, 1]
[1, 1, 1, 1]
[1, 1, 1, 1, 1]


而我们希望看到的应该是这样：

In [12]:
def f(x = None):
    if x is None:
        x = []
    x.append(1)
    return x

print(f())
print(f())
print(f())
print(f(x = [9,9,9]))
print(f())
print(f())

[1]
[1]
[1]
[9, 9, 9, 1]
[1]
[1]


## 高阶函数

以函数作为参数，或者返回一个函数的函数是高阶函数，常用的例子有 `map` 和 `filter` 函数：

`map(f, sq)` 函数将 `f` 作用到 `sq` 的每个元素上去，并返回结果组成的列表，相当于：
```python
[f(s) for s in sq]
```

注：python3中返回的不是列表，是一个迭代器，所以需要在外面加一层`list()`才会得到列表。

In [17]:
list(map(square, range(5)))

[0, 1, 4, 9, 16]

`filter(f, sq)` 函数的作用相当于，对于 `sq` 的每个元素 `s`，返回所有 `f(s)` 为 `True` 的 `s` 组成的列表，相当于：
```python
[s for s in sq if f(s)]
```

In [19]:
def is_even(x):
    return x % 2 == 0

list(filter(is_even, range(5)))

[0, 2, 4]

一起使用：

In [21]:
list(map(square, filter(is_even, range(5))))

[0, 4, 16]

`reduce(f, sq)` 函数接受一个二元操作函数 `f(x,y)`，并对于序列 `sq` 每次合并两个元素：

In [23]:
from functools import reduce
def my_add(x, y):
    return x + y

reduce(my_add, [1,2,3,4,5])

15

传入加法函数，相当于对序列求和。

返回一个函数：

In [24]:
def make_logger(target):
    def logger(data):
        with open(target, 'a') as f:
            f.write(data + '\n')
    return logger

foo_logger = make_logger('foo.txt')
foo_logger('Hello')
foo_logger('World')

In [30]:
import os
os.remove('foo.txt')

## 匿名函数

在使用 `map`， `filter`，`reduce` 等函数的时候，为了方便，对一些简单的函数，我们通常使用匿名函数的方式进行处理，其基本形式是：

    lambda <variables>: <expression>

例如，我们可以将这个：

In [32]:
print(list(map(square, range(5))))

[0, 1, 4, 9, 16]


用匿名函数替换为：

In [33]:
print(list(map(lambda x: x * x, range(5))))

[0, 1, 4, 9, 16]


匿名函数虽然写起来比较方便（省去了定义函数的烦恼），但是有时候会比较难于阅读：

In [34]:
s1 = reduce(lambda x, y: x+y, map(lambda x: x**2, range(1,10)))
print(s1)

285


当然，更简单地，我们可以写成这样：

In [35]:
s2 = sum(x**2 for x in range(1, 10))
print(s2)

285


# global 变量

一般来说，函数中是可以直接使用全局变量的值的：

In [36]:
x = 15

def print_x():
    print(x)
    
print_x()

15


但是要在函数中修改全局变量的值，需要加上 `global` 关键字：

In [37]:
x = 15

def print_newx():
    global x
    x = 18
    print(x)
    
print_newx()

print(x)

18
18


如果不加上这句 `global` 那么全局变量的值不会改变：

In [38]:
x = 15

def print_newx():
    x = 18
    print(x)
    
print_newx()

print(x)

18
15


## 递归

递归是指函数在执行的过程中调用了本身，一般用于分治法，不过在 `Python` 中这样的用法十分地小，所以一般不怎么使用：

Fibocacci 数列：

In [39]:
def fib1(n):
    """Fib with recursion."""

    # base case
    if n==0 or n==1:
        return 1
    # recurssive caae
    else:
        return fib1(n-1) + fib1(n-2)

print([fib1(i) for i in range(10)])

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]


一个更高效的非递归版本：

In [40]:
def fib2(n):
    """Fib without recursion."""
    a, b = 0, 1
    for i in range(1, n+1):
        a, b = b, a+b
    return b

print([fib2(i) for i in range(10)])

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]


速度比较：

In [41]:
%timeit fib1(20)
%timeit fib2(20)

4.42 ms ± 132 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
1.76 µs ± 75.1 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


对于第一个递归函数来说，调用 `fib(n+2)` 的时候计算 `fib(n+1), fib(n)`，调用 `fib(n+1)` 的时候也计算了一次 `fib(n)`，这样造成了重复计算。

使用缓存机制的递归版本，这里利用了默认参数可变的性质，构造了一个缓存：

In [42]:
def fib3(n, cache={0: 1, 1: 1}):
    """Fib with recursion and caching."""

    try:
        return cache[n]
    except KeyError:
        cache[n] = fib3(n-1) + fib3(n-2)
        return cache[n]

print ([fib3(i) for i in range(10)])

%timeit fib1(20)
%timeit fib2(20)
%timeit fib3(20)

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]
4.38 ms ± 69.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
1.74 µs ± 58.3 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
170 ns ± 6.95 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


# 迭代器

## 简介

迭代器对象可以在 `for` 循环中使用：

In [43]:
x = [2, 4, 6]

for n in x:
    print(n)

2
4
6


其好处是不需要对下标进行迭代，但是有些情况下，我们既希望获得下标，也希望获得对应的值，那么可以将迭代器传给 `enumerate` 函数，这样每次迭代都会返回一组 `(index, value)` 组成的元组：

In [44]:
x = [2, 4, 6]

for i, n in enumerate(x):
    print ('pos', i, 'is', n)

pos 0 is 2
pos 1 is 4
pos 2 is 6


迭代器对象必须实现 `__iter__` 方法：

In [45]:
x = [2, 4, 6]
i = x.__iter__()
print (i)

`__iter__()` 返回的对象支持 `next` 方法(`Python2`)，`__next__()`方法(`Python3`)。返回迭代器中的下一个元素：

In [50]:
print(i.__next__())

2


当下一个元素不存在时，会 `raise` 一个 `StopIteration` 错误：

In [51]:
print(i.__next__())
print(i.__next__())
print(i.__next__())

4
6


StopIteration: 

In [53]:
# 很多标准库函数返回的是迭代器：
r = reversed(x)
print (r)
#调用它的 next() 方法：
print(r.__next__())

6


字典对象的 `items` 方法返回的都是迭代器：
https://stackoverflow.com/questions/13998492/when-should-iteritems-be-used-instead-of-items

In [63]:
x = {'a':1, 'b':2, 'c':3}
i = x.items()
print (i)

dict_items([('a', 1), ('b', 2), ('c', 3)])


迭代器的 `__iter__` 方法返回它本身：

In [69]:
print(i.__iter__())

## 自定义迭代器

自定义一个 list 的取反迭代器：

In [72]:
class ReverseListIterator(object):
    
    def __init__(self, list):
        self.list = list
        self.index = len(list)
        
    def __iter__(self):
        return self
    
    def next(self):
        self.index -= 1
        if self.index >= 0:
            return self.list[self.index]
        else:
            raise StopIteration

In [80]:
x = [1,2,3]
y = ReverseListIterator(x)
y.next()

3

这里我们实现 [Collatz 猜想](http://baike.baidu.com/view/736196.htm)：

- 奇数 n：返回 3n + 1
- 偶数 n：返回 n / 2

直到 n 为 1 为止：

In [81]:
class Collatz(object):
    
    def __init__(self, start):
        self.value = start
        
    def __iter__(self):
        return self
    
    def next(self):
        if self.value == 1:
            raise StopIteration
        elif self.value % 2 == 0:
            self.value = self.value / 2
        else:
            self.value = 3 * self.value + 1
        return self.value

In [89]:
for i in range(7,1,-1):
    print(Collatz(i).next())

22
3.0
16
2.0
10
1.0


还没完全弄明白，之后填坑